# Walkthrough: Automatic Speech Recognition + Speaker Diarization

The purpose of this notebook is to provide a working example of inference workflow from [this blog post](https://huggingface.co/blog/asr-diarization) by the Hugging Face team.

## 1. Clone supporting files from Hub repo

In [ ]:
!git clone https://huggingface.co/sergeipetrov/asrdiarization-handler
!cp asrdiarization-handler/config.py asrdiarization-handler/requirements.txt asrdiarization-handler/diarization_utils.py .
!rm -rf asrdiarization-handler
!pip install -r requirements.txt

## 2. Download model artifacts to local dir

**Note:** This step is only here because it was requested to download model files separately to facilitate security scans.

In [1]:
import os
from pathlib import Path
from huggingface_hub import interpreter_login, snapshot_download

In [2]:
# login in to cache token
# could also use `notebook_login()`
# interpreter_login()

In [3]:
DIARIZATION_MODEL = "pyannote/speaker-diarization-3.1"
ASR_MODEL = "openai/whisper-large-v3"
ASSISTANT_MODEL = "distil-whisper/distil-large-v3"

In [8]:
MODELS_DIR = Path("models")
os.makedirs(MODELS_DIR, exist_ok=True)

In [12]:
for model in [ASR_MODEL, ASSISTANT_MODEL, DIARIZATION_MODEL]:
    snapshot_download(model, local_dir=MODELS_DIR / model.split("/")[1])

Fetching 21 files:   0%|          | 0/21 [00:00<?, ?it/s]

added_tokens.json:   0%|          | 0.00/34.6k [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/3.90k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/494k [00:00<?, ?B/s]

README.md:   0%|          | 0.00/15.2k [00:00<?, ?B/s]

...

(…)D.SpeakerDiarization.Benchmark.test.rttm:   0%|          | 0.00/648k [00:00<?, ?B/s]

(…)e.SpeakerDiarization.Benchmark.test.rttm:   0%|          | 0.00/1.98M [00:00<?, ?B/s]

(…)D.SpeakerDiarization.Benchmark.test.rttm:   0%|          | 0.00/3.60M [00:00<?, ?B/s]

(…)E.SpeakerDiarization.Benchmark.test.eval:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

(…)E.SpeakerDiarization.Benchmark.test.rttm:   0%|          | 0.00/1.44M [00:00<?, ?B/s]